In [1]:
sc

DSE has the spark context preconfigured....
usually this is boiler plate activity .. on a bare metal installation of spark etc..

In [2]:
sc.version

'2.1.2'

    RDD: Reselient Distributed dataset is an immutable data structure.. i.e. you cant change them once set.. think of array@java..?
    hence the multitude of examples that take one RDD and extract a subset of that RDD as a new RDD var and then work on it.. 
    retains the original RDD and the subset is smaller to work with ususally
    In Apache Spark, all work is done by creating new RDDs, transforming existing RDDs, or using RDDs to compute results. When working with RDDs, the Spark driver application automatically distributes the work across the cluster

In [3]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [4]:
x_nbr_rdd = sc.parallelize(x) 
# parallelize data - > RDD
# actions query RDD
# transformation : manipulate data

In [5]:
x_nbr_rdd.first()

1

In [6]:
x_nbr_rdd.take(3)


[1, 2, 3]

In [7]:
y = ["Hello Earthlings", "My Name is Spock"]

In [8]:
y_str_rdd = sc.parallelize(y)

In [9]:
y_str_rdd.take(1)

['Hello Earthlings']

In [10]:
x_nbr_rdd_2 = x_nbr_rdd.map(lambda x: x+1) # cool use of map rather than for :D

In [11]:
# see all elements: rdd_name.collect()
x_nbr_rdd_2.collect()

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

### collect throws up all the data.. even if there are a mn rows.. hence seldom seen in so examples

In [12]:
X = ["1,2,3,4,5,6,7,8,9,10"]
y_rd = sc.parallelize(X)

Sum_rd = y_rd.map(lambda y: y.split(",")).map(lambda y: (int(y[2])+int(y[9])))

In [13]:
Sum_rd.first()

13

In [14]:
Words = ["Hello Human. I'm Apache Spark and I love running analysis on data."]
words_rd = sc.parallelize(Words)
words_rd.first()

"Hello Human. I'm Apache Spark and I love running analysis on data."

In [15]:
words_rd2 = words_rd.map(lambda line: line.split(" "))
words_rd2.first()

['Hello',
 'Human.',
 "I'm",
 'Apache',
 'Spark',
 'and',
 'I',
 'love',
 'running',
 'analysis',
 'on',
 'data.']

In [16]:
words_rd2.count()


1

In [17]:
words_rd.count()

1

    The RDD still has only one element..

In [18]:
words_rd3 = words_rd.flatMap(lambda line: line.split(" "))
words_rd3.take(6)

['Hello', 'Human.', "I'm", 'Apache', 'Spark', 'and']

     flatmap is used to give each element of the split output .. its own element in the RDD...

In [19]:
z = ["First,Line", "Second,Line", "and,Third,Line"]
z_str_rdd = sc.parallelize(z)
z_str_rdd.first()

'First,Line'

In [20]:
z_str_rdd_split_flatmap = z_str_rdd.flatMap(lambda line: line.split(","))
z_str_rdd_split_flatmap.collect()

['First', 'Line', 'Second', 'Line', 'and', 'Third', 'Line']

In [21]:
countWords = z_str_rdd_split_flatmap.map(lambda word:(word,1))
countWords.collect()
# creating a tally of the words

[('First', 1),
 ('Line', 1),
 ('Second', 1),
 ('Line', 1),
 ('and', 1),
 ('Third', 1),
 ('Line', 1)]

In [22]:
#using the 1 marked against each owrd for aggregation
from operator import add
countWords2 = countWords.reduceByKey(add)
countWords2.collect()

[('Line', 3), ('Second', 1), ('Third', 1), ('First', 1), ('and', 1)]

In [23]:
#filtering .. filters the said term
words_rd3 = z_str_rdd_split_flatmap.filter(lambda line: "Second" in line) # this rdd .. created above

print ("The count of words " + str(words_rd3.first())) # use brackets... pn py 3 print is a funciton and not a statement
print ("Is: " + str(words_rd3.count()))

The count of words Second
Is: 1


In [24]:
!rm README.md* -f # ! for sys operations 
# remove thereadme fileif it already exists
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md

--2018-01-19 00:33:15--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/README.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3624 (3.5K) [text/plain]
Saving to: ‘README.md’

100%[======================================>] 3,624       --.-K/s   in 0s      

2018-01-19 00:33:15 (26.7 MB/s) - ‘README.md’ saved [3624/3624]



In [25]:
textfile_rdd = sc.textFile("README.md")
textfile_rdd.count()

98

In [26]:
Spark_lines = textfile_rdd.filter(lambda line: "Spark" in line)
Spark_lines.first() # FILTER FOR A LINE AND DISPLAY FIRST

'# Apache Spark'

In [27]:
print ("The file README.md has " + str(Spark_lines.count()) + \
" of " + str(textfile_rdd.count()) + \
" Lines with word Spark in it.")

The file README.md has 19 of 98 Lines with word Spark in it.


    Run a flatMap transformation on the Spark_lines RDD and split on white spaces.
    Create an RDD with key-value pairs where the first element of the tuple is the word and the second element is the number 1.
    Run a reduceByKey method with the add function to count the number of instances of each word.
    Filter the resulting RDD to keep only the elements that start with the word "Spark". In Python, the syntax to determine whether a string starts with a token is: string.startswith("token")
    Display the resulting list of elements that start with "Spark".

In [28]:
temp = Spark_lines.flatMap(lambda line:line.split(" ")).map(lambda x:(x,1)).reduceByKey(add)
filter_fn = lambda x_y: x_y[0].startswith("Spark")
# SO : 27776026
# Syntax issues transtitionaing from 2 -> 3
temp.filter(filter_fn).collect()

[('Spark](#building-spark).', 1),
 ('Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1),
 ('Spark.', 1),
 ('Spark', 14),
 ('SparkPi', 2)]

In [29]:
filter_fn = lambda x_y: "Spark" in x_y[0] # general syntax for python 3.x a string contains a certain token
temp.filter(filter_fn).collect()

[('Spark](#building-spark).', 1),
 ('Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1),
 ('Spark.', 1),
 ('Spark', 14),
 ('SparkPi', 2),
 ('tests](https://cwiki.apache.org/confluence/display/SPARK/Contributing+to+Spark#ContributingtoSpark-AutomatedTesting).',
  1)]

### now we try with some real data


In [30]:
!rm Scores.txt* -f
!wget https://raw.githubusercontent.com/carloapp2/SparkPOT/master/Scores.txt
 
Raw_Rdd = sc.textFile("Scores.txt")

SumScores = Raw_Rdd.map(lambda l: l.split(",")).\
map(lambda v : (v[0], int(v[1])+int(v[2])+int(v[3])+int(v[4])))

Final = SumScores.map(lambda avg: (avg[0],avg[1],avg[1]/4.0))

Final.take(5)

--2018-01-19 00:33:16--  https://raw.githubusercontent.com/carloapp2/SparkPOT/master/Scores.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75 [text/plain]
Saving to: ‘Scores.txt’

100%[======================================>] 75          --.-K/s   in 0s      

2018-01-19 00:33:17 (9.13 MB/s) - ‘Scores.txt’ saved [75/75]



[('Carlo', 15, 3.75),
 ('Mokhtar', 15, 3.75),
 ('Jacques', 15, 3.75),
 ('Braden', 15, 3.75),
 ('Chris', 15, 3.75)]